In [2]:
import tensorflow as tf
import pandas as pd

tf.test.is_built_with_cuda()

2023-09-21 14:07:31.198714: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-21 14:07:32.932390: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


True

In [1]:
dataset = pd.read_csv(r'Dataset.csv')
print(dataset)

NameError: name 'pd' is not defined